# Visualization: Trading Session

In [14]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [15]:
from trading_bot.agent import Agent

model_name = 'ddqn_UJ_50'
test_stock = 'data/USD_JPY_TEST.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [16]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual'})
# convert dates from object to DateTime type
dates = df['Date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['Date'] = dates

df.head()

,Date,actual
0,2020-01-01,108.70
1,2020-01-02,108.58
2,2020-01-03,108.09
3,2020-01-06,108.38
4,2020-01-07,108.44


### 3. Running Eval

In [17]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG switching to TensorFlow for CPU
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $108.70
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Sell at: $108.58 | Position: -$0.1200
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $109.14
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Sell at: $109.47 | Position: +$0.3300
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $109.98
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $109.89
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $110.16
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $110.16
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Sell at: $109.84 | Position: -$0.1400
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $109.28
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $109.14
2020-03-28 16:30:15 DESKTOP-UBSIEG2 root[8160] DEBUG Buy at: $108.69
2020-03-28 16:30:15 D

### 4. Visualize the Double DQ Learning

In [18]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    df['size']='Buy/Sell'
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])), max(max(df['actual']), max(df['position']))), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='Date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('Date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action', size='size'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=800)
    
    return chart

In [19]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [20]:
df.to_csv("ddqn-USD_JPY.csv")